# Flinkster car usage

One of the advantages of carsharing over carowning is supposed to be the higher usage rate. There is an existing [(and critizied) study](http://www.spiegel.de/auto/aktuell/car2go-und-drivenow-studie-kritisiert-carsharing-von-bmw-und-mercedes-a-987998.html) that looks at the usage for the free-floating offers by Car2Go and DriveNow. It has been crititzied because it does not use official data but scraped data instead.

I wanted to also take a look at the data that exists publicly. I know that this is the case for the Detusche Bahn carsharing provider Flinkster and that the data is published as open data.

So I downloaded the rides from the [official dataportal](http://data.deutschebahn.com/dataset/data-flinkster/resource/26001192-b798-44f0-9d86-bfbcbac12f40) and ran some analyses on it:


## Data Loading
First we need to load the data:

In [30]:
from io import BytesIO, StringIO
from zipfile import ZipFile

import pandas as pd
import numpy as np
import requests
import requests

URL = "http://download-data.deutschebahn.com/static/datasets/flinkster/20170516/OPENDATA_BOOKING_CARSHARING.zip"

def get_data():
    ziped_data = requests.get(URL)
    with ZipFile(BytesIO(ziped_data.content)) as infile:
        data = infile.read("OPENDATA_BOOKING_CARSHARING.csv")
        return StringIO(data.decode("utf-8"))

data = get_data()
    
df = pd.read_csv(data,
                 delimiter=";",
                 parse_dates=["DATE_FROM", "DATE_UNTIL"])


## Analysis
Afterwards we group the data by car and calculate the total time that this car is in the database vs the time that it has been booked for every entry in the data:

In [32]:
results = []
for (index, group) in df.groupby("VEHICLE_HAL_ID"):
    total_time = group["DATE_UNTIL"].max() - group["DATE_FROM"].min()
    usage_time = (group["DATE_UNTIL"] - group["DATE_FROM"]).sum()
    results.append({"id": index, "percentage_used": usage_time/total_time})
    
results_df = pd.DataFrame(results)


## Results
Lets looks at the results:

In [33]:
results_df.head()

,id,percentage_used
0,143031,0.261484
1,146233,0.196357
2,146236,0.131040
3,146243,0.064894
4,146272,0.173471


We now have a DataFrame that contains the ids and their respective usage times


You can check the records for an individual car to see if the results make sense:

In [34]:
df[df["VEHICLE_HAL_ID"] == 146243]

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,START_RENTAL_ZONE,START_RENTAL_ZONE_HAL_ID,END_RENTAL_ZONE,END_RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL
1366,21373142,100001,146243,FBC6FB200E2F9692251F30492C28C35DF1C5B47F,2014-01-02 09:36:19,2014-01-02 10:00:00,2014-01-02 13:30:00,Nein,Ja,15.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
2401,21389857,100001,146243,FAB25A786E9B5A1622036C6F97FD8052AC9468A2,2014-01-04 00:41:48,2014-01-04 00:45:00,2014-01-04 02:15:00,Nein,Ja,11.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
3939,21416514,100001,146243,0BD3EA10ABD67828D190CEA23122800599FD0104,2014-01-07 09:35:23,2014-01-07 18:30:00,2014-01-08 08:30:00,Nein,Nein,27.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
3968,21417106,100001,146243,E8AA508408ADC338F093B1D4474F9489E1B4C027,2014-01-07 10:21:42,2014-01-07 12:30:00,2014-01-07 13:15:00,Nein,Nein,14.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
4470,21427867,100001,146243,E8AA508408ADC338F093B1D4474F9489E1B4C027,2014-01-08 09:20:24,2014-01-08 12:15:00,2014-01-08 13:00:00,Nein,Nein,13.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
4584,21430059,100001,146243,0BD3EA10ABD67828D190CEA23122800599FD0104,2014-01-08 12:48:36,2014-01-08 18:15:00,2014-01-09 08:30:00,Nein,Nein,33.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
5506,21450582,100001,146243,121CED2580A6461D987C4BA3373D32B6FE32A54A,2014-01-10 09:55:05,2014-01-10 09:45:00,2014-01-10 11:00:00,Nein,Nein,11.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
5750,21454234,100001,146243,047A5E3F716A50E25A04646CA98F62DC232DBDA0,2014-01-10 15:43:58,2014-01-11 11:00:00,2014-01-11 12:30:00,Nein,Nein,11.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
6370,21463126,100001,146243,047A5E3F716A50E25A04646CA98F62DC232DBDA0,2014-01-11 16:29:30,2014-01-11 17:45:00,2014-01-11 18:15:00,Nein,Nein,6.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet
7332,21479656,100001,146243,137EF9F5CFC37021AC71C149B27D5E4FA557D79F,2014-01-13 16:55:14,2014-01-13 17:15:00,2014-01-13 18:15:00,Nein,Nein,11.0,GSG-HOF Helmholtzstraße,403167,GSG-HOF Helmholtzstraße,403167,Station,Berlin,Internet


And finally the question that we want to see answered: How long are the cars booked on average:

In [36]:
results_df["percentage_used"].median()

0.2533048551974917

## Conclusion

So the answer to our question is the following: Flinkster cars are rented for around 25% of the total time.
This of course does not mean that they are also driven for the whole time but it at least gives us a staring point.

🚗